In [1]:
import cv2
from PIL import Image
import numpy as np 
from matplotlib import pyplot as plt

In [182]:
srcImg = cv2.imread("/home/melt61/TensorflowProjects/dcscn-super-resolution/char_test/rotImg_04.jpg"
                   ,0)

In [183]:
opWidth = cv2.getOptimalDFTSize(srcImg.shape[1])
opHeight = cv2.getOptimalDFTSize(srcImg.shape[0])
padded = cv2.copyMakeBorder(srcImg, 0, opWidth-srcImg.shape[1], 0, opHeight-srcImg.shape[0], 
                            cv2.BORDER_CONSTANT, value = (0,0,0))

In [184]:
tfImg = np.fft.fft2(padded)
shImg = np.fft.fftshift(tfImg)
fft_result_Img = 20 * np.log(np.abs(shImg))

In [123]:
#dftImg = cv2.dft(np.float32(padded), flags=cv2.DFT_COMPLEX_OUTPUT)
#dft_shift = np.fft.fftshift(dftImg)
#magnitude_2 = 20 * np.log(cv2.magnitude(dft_shift[:,:,0], dft_shift[:,:,1]))  

In [159]:
#tt1 = Image.fromarray(magnitude_2)
tt1 = Image.fromarray(fft_result_Img)
tt1.show()


In [185]:
#fft_gray = cv2.cvtColor(tt1, cv2.COLOR_BGR2GRAY)
tImg = fft_result_Img.astype(np.uint8)
#edges = cv2.Canny(fft_result_Img, 50, 150, apertureSize= 3 )
#minLineLength = 10
#maxLineGap = 50

ret, biImg = cv2.threshold(tImg, 40, 255, cv2.THRESH_BINARY_INV)

lines = cv2.HoughLines(biImg, 1, np.pi/180, 180)

In [186]:
linesnum = lines.shape[0]
lineImg = np.zeros(srcImg.shape)


In [187]:
for l in range(linesnum):
    for rho,theta in lines[l]:
        #if 0 < theta < 1/2  or 1/2 < theta < 1:
        #   print(theta/np.pi)
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(lineImg, (x1, y1), (x2, y2), (255, 0, 0), 2)

In [188]:
tt = Image.fromarray(padded)
tt.show()
tt.save("/home/melt61/Rotation_Rectified/rotImg_04_padded.png")

tt1 = Image.fromarray(fft_result_Img)
tt1.show()
tt1 = tt1.convert('RGB')
tt1.save("/home/melt61/Rotation_Rectified/rotImg_04_fft.png")

tt2 = Image.fromarray(biImg)
tt2.show()
tt2 = tt2.convert('RGB')
tt2.save("/home/melt61/Rotation_Rectified/rotImg_04_bi.png")

tt3 = Image.fromarray(lineImg)
tt3.show()
tt3 = tt3.convert('RGB')
tt3.save("/home/melt61/Rotation_Rectified/rotImg_04_lines.png")

In [189]:
angle = 0.0
piThresh = np.pi/90
for i in range(linesnum):
    rhoC,thetaC = lines[i][0]
    #print(lines[i])
    if np.abs(thetaC) < piThresh or np.abs(thetaC-np.pi/2) < piThresh:
        continue
    else:    
        print(thetaC)
        angle = thetaC
        break

0.139626


In [190]:
angle if angle < np.pi/2 else angle-np.pi

if angle != np.pi/2:
    angleT = srcImg.shape[0] * np.tan(angle) / srcImg.shape[1]
    angle = np.arctan(angleT)
    print(angle)

rows,cols = srcImg.shape
RotationM = cv2.getRotationMatrix2D((cols/2, rows/2), angle/np.pi*180, 1)
dst = cv2.warpAffine(srcImg, RotationM, (cols, rows))

0.192390862344


In [191]:
tt4 = Image.fromarray(dst)
tt4.show()

In [192]:
tt4.save("/home/melt61/Rotation_Rectified/aRota_04.jpg",dpi=(150, 150))